In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import holidays
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [6]:

def load_and_preprocess_data(file_path='../../public/data/pjm_dataset/pjm_hourly_est.csv'):
    """
    Load and perform initial preprocessing of PJM data
    """
    # Read data
    df = pd.read_csv(file_path, parse_dates=['Datetime'])
    
    # Sort by datetime to ensure temporal order
    df = df.sort_values('Datetime')
    
    # Filter date range to where we have most complete data (based on EDA)
    df = df[(df['Datetime'] >= '2002-04-01') & (df['Datetime'] <= '2018-08-03')]
    
    # Focus on PJME and add PJMW as a feature
    df = df[['Datetime', 'PJME', 'PJMW']]
    
    # Handle the one missing value in PJMW
    df['PJMW'] = df['PJMW'].fillna(method='ffill')
    
    # Print data info for debugging
    print("Initial data shape:", df.shape)
    print("\nMissing values in initial data:")
    print(df.isnull().sum())
    
    return df

def create_temporal_features(df):
    """
    Create time-based features identified in EDA
    """
    df = df.copy()
    
    # Basic time features
    df['hour'] = df['Datetime'].dt.hour
    df['day'] = df['Datetime'].dt.day
    df['month'] = df['Datetime'].dt.month
    df['year'] = df['Datetime'].dt.year
    df['dayofweek'] = df['Datetime'].dt.dayofweek
    df['quarter'] = df['Datetime'].dt.quarter
    df['weekofyear'] = df['Datetime'].dt.isocalendar().week
    
    # Cyclical encoding of time features
    # This helps capture the cyclical nature of time features
    df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek']/7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek']/7)
    
    # Is weekend feature
    df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
    
    # Time of day categories (based on load patterns from EDA)
    df['timeofday'] = pd.cut(df['hour'], 
                            bins=[-1, 6, 12, 17, 23],
                            labels=['night', 'morning', 'afternoon', 'evening'])
    
    return df

def create_holiday_features(df):
    """
    Add holiday-related features
    """
    us_holidays = holidays.US()
    
    df['is_holiday'] = df['Datetime'].dt.date.apply(
        lambda x: 1 if x in us_holidays else 0
    )
    
    # Add features for days before/after holidays
    df['is_day_before_holiday'] = df['Datetime'].dt.date.apply(
        lambda x: 1 if (x + timedelta(days=1)) in us_holidays else 0
    )
    df['is_day_after_holiday'] = df['Datetime'].dt.date.apply(
        lambda x: 1 if (x - timedelta(days=1)) in us_holidays else 0
    )
    
    return df

def create_lagged_features(df, target_col='PJME', lags=[1, 24, 48, 168]):
    """
    Create lagged features based on target variable
    lags: list of hour offsets for which to create lags
    """
    df = df.copy()
    
    for lag in lags:
        df[f'{target_col}_lag_{lag}h'] = df[target_col].shift(lag)
    
    return df

def create_rolling_features(df, target_col='PJME', 
                          windows=[24, 168], stats=['mean', 'std']):
    """
    Create rolling window features
    windows: list of hour windows for which to create features
    stats: list of statistics to compute for each window
    """
    df = df.copy()
    
    for window in windows:
        for stat in stats:
            df[f'{target_col}_roll_{window}h_{stat}'] = getattr(
                df[target_col].rolling(window, min_periods=1), stat
            )()
    
    return df

def handle_missing_values(df):
    """
    Handle missing values in features with a more robust approach
    """
    df = df.copy()
    
    # 1. Handle temporal features (these shouldn't have missing values)
    time_cols = ['hour', 'day', 'month', 'year', 'dayofweek', 'quarter', 'weekofyear',
                'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'dayofweek_sin', 
                'dayofweek_cos', 'is_weekend']
    
    # 2. Handle holiday features
    holiday_cols = ['is_holiday', 'is_day_before_holiday', 'is_day_after_holiday']
    df[holiday_cols] = df[holiday_cols].fillna(0)
    
    # 3. Handle lagged and rolling features
    lag_roll_cols = [col for col in df.columns if 'lag' in col or 'roll' in col]
    
    # First forward fill
    df[lag_roll_cols] = df[lag_roll_cols].fillna(method='ffill')
    
    # Then backward fill any remaining NAs
    df[lag_roll_cols] = df[lag_roll_cols].fillna(method='bfill')
    
    # 4. For any remaining missing values in numeric columns, fill with median
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    for col in numeric_cols:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].median())
            
    # 5. For categorical columns, fill with mode
    cat_cols = ['timeofday']
    for col in cat_cols:
        if col in df.columns and df[col].isnull().any():
            df[col] = df[col].fillna(df[col].mode()[0])
    
    return df

def scale_features(df, exclude_cols=['Datetime', 'PJME', 'timeofday']):
    """
    Scale numerical features
    """
    # Identify columns to scale
    cols_to_scale = [col for col in df.columns 
                    if col not in exclude_cols and df[col].dtype in ['int64', 'float64']]
    
    # Scale features
    scaler = StandardScaler()
    df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
    
    return df, scaler

def prepare_features(df):
    """
    Main function to prepare all features
    """
    # Create all feature groups
    df = create_temporal_features(df)
    df = create_holiday_features(df)
    df = create_lagged_features(df)
    df = create_rolling_features(df)
    
    # Handle missing values
    df = handle_missing_values(df)
    
    # Scale features
    df, scaler = scale_features(df)
    
    return df, scaler

def split_data(df, test_size=0.2):
    """
    Split data into training and testing sets, respecting temporal order
    """
    # Calculate split point
    split_idx = int(len(df) * (1 - test_size))
    
    # Split the data
    train = df.iloc[:split_idx]
    test = df.iloc[split_idx:]
    
    return train, test


def validate_features(df_featured):
    """
    Validate the engineered features
    """
    # Check for missing values by column
    missing = df_featured.isnull().sum()
    print("\nMissing values by column:")
    print(missing[missing > 0])
    
    # Check feature correlations with target, excluding non-numeric columns
    numeric_df = df_featured.select_dtypes(include=['float64', 'int64'])
    correlations = numeric_df.corr()['PJME'].sort_values(ascending=False)
    print("\nTop 10 feature correlations with PJME:")
    print(correlations[:10])
    
    # Check feature value ranges
    print("\nFeature ranges:")
    print(numeric_df.describe())
    
    return missing, correlations

In [8]:
# Load data
df = load_and_preprocess_data()

# Prepare features
df_featured, scaler = prepare_features(df)

# Basic validation
print("Dataset shape:", df_featured.shape)
print("\nFeatures created:", [col for col in df_featured.columns if col not in ['Datetime', 'PJME']])
print("\nMissing values:", df_featured.isnull().sum().sum())

# Validate features
missing, correlations = validate_features(df_featured)

# Split data
train_data, test_data = split_data(df_featured)
print("\nTrain shape:", train_data.shape)
print("Test shape:", test_data.shape)
    
# Save processed data
train_data.to_csv('../../data/processed/train_data.csv', index=False)
test_data.to_csv('../../data/processed/test_data.csv', index=False)
print("\nProcessed data saved to CSV files")


Initial data shape: (143207, 3)

Missing values in initial data:
Datetime    0
PJME        0
PJMW        1
dtype: int64
Dataset shape: (143207, 29)

Features created: ['PJMW', 'hour', 'day', 'month', 'year', 'dayofweek', 'quarter', 'weekofyear', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'dayofweek_sin', 'dayofweek_cos', 'is_weekend', 'timeofday', 'is_holiday', 'is_day_before_holiday', 'is_day_after_holiday', 'PJME_lag_1h', 'PJME_lag_24h', 'PJME_lag_48h', 'PJME_lag_168h', 'PJME_roll_24h_mean', 'PJME_roll_24h_std', 'PJME_roll_168h_mean', 'PJME_roll_168h_std']

Missing values: 0

Missing values by column:
Series([], dtype: int64)

Top 10 feature correlations with PJME:
PJME                   1.000000
PJME_lag_1h            0.974913
PJME_lag_24h           0.891863
PJMW                   0.875714
PJME_lag_168h          0.781288
PJME_lag_48h           0.773443
PJME_roll_24h_mean     0.702885
PJME_roll_168h_mean    0.566898
PJME_roll_24h_std      0.490923
PJME_roll_168h_std     0.4085